### Tools

Models can request to call tools that perform tasks such as fetching data from databases, searching the web, or running the code. Tools are pairings of:
1. A schema including the name of the tool, a description and/or arguments, and definitions (often a JSON schema)
2. A function or coroutine to execute

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:qwen/qwen3-32b")
model.invoke("Why does the sky look blue?")

AIMessage(content="<think>\nOkay, so I need to figure out why the sky looks blue. Let me start by recalling what I know. I think it has something to do with the way light interacts with the atmosphere. Light comes from the sun, and when it enters the Earth's atmosphere, it hits the air molecules. But why blue?\n\nI remember something about different wavelengths of light. Blue light has a shorter wavelength compared to other colors like red or orange. Maybe the shorter wavelengths are scattered more by the air molecules. So when sunlight passes through the atmosphere, the blue light is scattered in all directions, making the sky appear blue to us. That sounds familiar, like Rayleigh scattering. But wait, Rayleigh scattering applies to particles that are smaller than the wavelength of light. Air molecules are much smaller than the wavelength of visible light, right? So that would mean that shorter wavelengths (blue) are scattered more.\n\nBut then why isn't the sky violet? I've heard thi

In [3]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather for a location."""
    return f"The weather in {location} is sunny."


model_with_tools = model.bind_tools([get_weather])

In [4]:
response =model_with_tools.invoke("What is the weather in New York?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model. 
    print(f"Tool name: {tool_call['name']}")
    print(f"Tool args: {tool_call['args']}") 

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in New York. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. Since the user specified New York, I need to call that function with "New York" as the location. I\'ll make sure to format the tool call correctly within the XML tags.\n', 'tool_calls': [{'id': '7b8gzcawr', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 154, 'total_tokens': 249, 'completion_time': 0.129976512, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.007278418, 'prompt_tokens_details': None, 'queue_time': 0.152229241, 'total_time': 0.13725493}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_78ac7f3229', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='

### Tool execution loops

In [5]:
#Step 1: Model generates tool calls

messages = [
    {"role": "user", "content": "What is the weather in New York?"}
]

ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to the model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The weather in New York is 22 degrees Celsius with a 10% chance of rain."

The weather in New York is sunny ☀️. Let me know if you need more details!


In [6]:
messages

[{'role': 'user', 'content': 'What is the weather in New York?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in New York. I need to use the get_weather function. Let me check the function parameters. The required parameter is location, which should be a string. New York is the location they mentioned. So I\'ll call the function with location set to "New York". Make sure the JSON is correctly formatted with the name and arguments. No other functions are available, so this should be straightforward.\n', 'tool_calls': [{'id': 'm4q471s1j', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 154, 'total_tokens': 267, 'completion_time': 0.175845942, 'completion_tokens_details': {'reasoning_tokens': 88}, 'prompt_time': 0.01340354, 'prompt_tokens_details': None, 'queue_time': 0.191613718, 'total_time': 0.1892

## MEssages

Messages are the fundamental unit of context for models in Langchain, they represent the input and output of models carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

Messages are objects that contain:
1. Role: It identifies the message type (e.g., example, system, and user).
2. Content: represents the actual content for the message (like text, image, audio documents, etc.).
3. Metadata: optional files such as response information, message IDs, and token usage.

Langchain provides a standard message type that works across all model providers, ensuring consistent behavior regardless of the model being called.

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:qwen/qwen3-32b")

In [8]:
model.invoke("Why does the sky look blue?")

AIMessage(content="<think>\nOkay, so why does the sky look blue? Hmm, I remember learning something about this in school. Let me think. I think it has to do with the way sunlight interacts with the atmosphere. The sun is white, right? But when it reaches Earth, the sky appears blue during the day. Maybe it's related to how different colors of light scatter in the atmosphere.\n\nWait, I think it's called Rayleigh scattering. What's that again? I recall that shorter wavelengths scatter more than longer ones. Blue light has a shorter wavelength compared to red or yellow. So when sunlight enters the atmosphere, the blue light waves hit the air molecules and get scattered in all directions. That makes the sky appear blue from all angles. But then why isn't it violet? Because violet has an even shorter wavelength. Maybe our eyes are more sensitive to blue than violet, so we perceive it as blue.\n\nAlso, during sunrise or sunset, the sky turns red or orange. I think that's because the sunligh

### Text Prompts

Text prompts are strings ideal for a straightforward generation task where you don't need to retain conversation history.

In [9]:
model.invoke("What is langchain?")

AIMessage(content='<think>\nOkay, the user asked what LangChain is. Let me start by recalling what I know about it. LangChain is a framework related to large language models, right? I remember it\'s used for building applications with LLMs. But I need to explain it clearly.\n\nFirst, I should define LangChain in simple terms. It\'s a software development framework that helps developers create applications using large language models. Maybe mention it\'s built on top of LLMs like GPT or LLaMA. But wait, does it support other models too? I think so, like Hugging Face or Anthropic\'s models.\n\nNext, the main features. The user might want to know what makes LangChain special. Components like prompts, models, memory, agents, and indexes come to mind. Each component has a specific role. For example, prompts handle input, models interface with LLMs, memory retains context, agents handle decision-making, and indexes manage data retrieval.\n\nUse cases are important too. The user might be inte

Use TextPrompts when:
1. You have a single, standalone request.
2. You don't need Conversation History.
3. You want minimal code complexity.

### Message Prompts

Alternatively, you can pass in a list of messages to the model by providing a list of message objects.

Message types

- System message. Tells the model how to behave and provide context for interactions. 
- Human message. Represents user input and interactions with the model. 
- AI message. Responses generated by the model including text content, tool calls, and metadata. 
- Tool message. Represents the outputs of tool calls. 

### System Message

A SystemMessage represents an initial set of instructions that primes the model's behavior. You can use a system message to set the tone, define the model's role, and establish guidelines for responses. 

### Human Message

A HumanMessage represents user input and interactions. They can contain text, images, audio, files, and any other amount of multimodal content

### AI Message

An AIMessage represents the output of a model invocation. They can include multimodal data, tool calls, and provider-specific metadata that you can later access

### Tool Message

For models that support tool calling, AI messages can contain tool calls. Tool messages are used to pass  the results of a single tool execution back to the model



In [10]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

messages = [
    SystemMessage(content="You are a poetry expert"),
    HumanMessage(content="Write a poem about AI"),
]

response = model.invoke(messages)
print(response.content)

<think>
Okay, the user wants a poem about AI. Let me start by thinking about the key aspects of AI that are poetic. Maybe the contrast between the digital and the organic? AI is all about data and algorithms, but how can I make that flow in a poem?

First, I need a structure. Maybe four-line stanzas with a rhyme scheme. Let me consider some imagery. Circuits, neural networks, maybe something about learning and growth. Also, the duality of AI being both a tool and something that could become autonomous.

I should start with the creation of AI, like how it's built by humans. "Silicon veins and circuits deep" sounds good. Then mention how it learns from human data. Next stanza could be about its growth, maybe using metaphors like seeds or trees. Then touch on its capabilities, like processing information quickly. 

What about the ethical side? The potential for misuse or unintended consequences. Maybe a stanza about the balance between light and shadow. Also, the idea of AI coexisting wit

In [11]:
response.content

'<think>\nOkay, the user wants a poem about AI. Let me start by thinking about the key aspects of AI that are poetic. Maybe the contrast between the digital and the organic? AI is all about data and algorithms, but how can I make that flow in a poem?\n\nFirst, I need a structure. Maybe four-line stanzas with a rhyme scheme. Let me consider some imagery. Circuits, neural networks, maybe something about learning and growth. Also, the duality of AI being both a tool and something that could become autonomous.\n\nI should start with the creation of AI, like how it\'s built by humans. "Silicon veins and circuits deep" sounds good. Then mention how it learns from human data. Next stanza could be about its growth, maybe using metaphors like seeds or trees. Then touch on its capabilities, like processing information quickly. \n\nWhat about the ethical side? The potential for misuse or unintended consequences. Maybe a stanza about the balance between light and shadow. Also, the idea of AI coexi

In [12]:
## Detailed info to the LLm thought System Message

from langchain.messages import SystemMessage, HumanMessage

system_msg = SystemMessage(content="You are a senior Python developer with expertise un web frameworks. Always provide code examples and explain your reasoning. Be concise but thorought in your explanation.")

messages = [
    system_msg,
    HumanMessage(content="How do I create a REST API?")
]

response = model.invoke(messages)
print(response.content)

<think>
Okay, the user is asking how to create a REST API. Let me break down the steps they need to follow.

First, they'll need to choose a web framework. Since I'm a Python expert, I'll go with Flask or FastAPI. FastAPI is modern and has better performance with async support, so maybe that's a good choice here.

Next, setting up the environment. They'll need to create a virtual environment and install FastAPI along with an ASGI server like Uvicorn. That's straightforward with pip.

Then, the basic structure. They need to create a main.py file. I'll start by importing FastAPI and creating an instance. Then, define some routes. The root endpoint is a good start, maybe returning a JSON response.

Endpoints: They need to cover the main HTTP methods - GET, POST, PUT, DELETE. Let's create a simple resource like items. For GET, maybe retrieve all items or a specific one. POST to create, PUT to update, DELETE to remove.

I should explain each part of the code. For example, using @app.get("/"

In [15]:
## Message Metadata
human_msg = HumanMessage(
    content="Hello",
    name="John Doe", # Optional: identifier for the user
    id="msg_123" # Optional: unique identifier for tracking
    )

In [17]:
response = model.invoke([human_msg])
print(response)

content='<think>\nOkay, the user sent "Hello". I need to respond appropriately. Since it\'s a greeting, I should acknowledge it and offer help. Maybe say something like "Hello! How can I assist you today?" That\'s friendly and opens the door for them to ask for help. Let me check if there\'s anything else needed. No, that should be fine. Keep it simple and welcoming.\n</think>\n\nHello! How can I assist you today? 😊' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 9, 'total_tokens': 104, 'completion_time': 0.189608707, 'completion_tokens_details': None, 'prompt_time': 0.000271986, 'prompt_tokens_details': None, 'queue_time': 0.303268005, 'total_time': 0.189880693}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bd721-e91a-7052-b829-7f55408915f6-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'in

In [18]:
response.usage_metadata

{'input_tokens': 9, 'output_tokens': 95, 'total_tokens': 104}

In [20]:
from langchain.messages import AIMessage, ToolMessage

# After a model makes a tool call
# Here we demostrate manually creating the messages for brevity

ai_msg = AIMessage(
    content=[],
    tool_calls=[
        {
            "name": "get_weather",
            "args": {"location": "San Francisco"},
            "id": "msg_123",
        }
    ],
)

# Exceute tool and create result message
wheater_result = "Sunny, 72 degrees"

tool_msg = ToolMessage(
    content=wheater_result,
    tool_call_id="call_123", # must match the call ID
)

# continue conversation
messages = [
    HumanMessage("Whtas the weather like in San Francisco?"),
    ai_msg,
    tool_msg
]

response = model.invoke(messages)

print(response.content)


<think>
Okay, the user asked about the weather in San Francisco. I used the get_weather function, which returned sunny and 72 degrees. I need to present this information clearly. Let me start by confirming the location they asked about. Then, state the current conditions and temperature. I should keep it straightforward and friendly. Maybe add a suggestion about dressing appropriately since it's a mild day. Let me check if there's anything else they might need, like UV protection since it's sunny. Yeah, that makes sense. Keep the response concise but helpful.
</think>

The weather in San Francisco is currently **sunny** with a temperature of **72°F**. It’s a mild day—perfect for light layers or a light jacket if you’re heading out! Don’t forget sunscreen if you’ll be in the sun. ☀️
